In [2]:
import json
import os.path as osp
import sys
import time

import joblib
import numpy as np
import matplotlib
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from scipy.spatial import ConvexHull
import tensorflow as tf

from sandbox.embed2learn.envs.util import colormap_mpl

In [3]:
def rollout(env,
            agent,
            z,
            max_path_length=np.inf,
            animated=False,
            speedup=1,
            always_return_paths=False):

    observations = []
    tasks = []
    latents = []
    latent_infos = []
    actions = []
    rewards = []
    agent_infos = []
    env_infos = []

    # Resets
    o = env.reset()
    agent.reset()

    # Sample embedding network
    # NOTE: it is important to do this _once per rollout_, not once per
    # timestep, since we need correlated noise.
    # t = env.active_task_one_hot
    # z, latent_info = agent.get_latent(t)

    if animated:
        env.render()

    path_length = 0
    while path_length < max_path_length:
        a, agent_info = agent.get_action_from_latent(z, o)
        next_o, r, d, env_info = env.step(a)
        observations.append(agent.observation_space.flatten(o))
        path_length += 1
        if d:
            break
        o = next_o
        if animated:
            env.render()
            timestep = 0.05
            time.sleep(timestep / speedup)
    if animated and not always_return_paths:
        return

    return np.array(observations)

def rollout_given_z(env,
                    agent,
                    z,
                    max_path_length=np.inf,
                    animated=False,
                    speedup=1):
    o = env.reset()
    agent.reset()

    if animated:
        env.render()

    path_length = 0
    observations = []
    while path_length < max_path_length:
        a, agent_info = agent.get_action_from_latent(o, z)
        next_o, r, d, env_info = env.step(a)
        observations.append(agent.observation_space.flatten(o))
        path_length += 1
        if d:
            break
        o = next_o
        if animated:
            env.render()
            time.sleep(0.05 / speedup)

    return np.array(observations)


def get_z_dist(t, policy):
    """ Get the latent distribution for a task """
    onehot = np.zeros(policy.task_space.shape, dtype=np.float32)
    onehot[t] = 1
    _, latent_info = policy.get_latent(onehot)
    return latent_info["mean"], np.exp(latent_info["log_std"])

In [ ]:
from scipy.spatial import ConvexHull

xs = np.array([0., 0., 2., 2.])
ys = np.array([0., 2., 2., 0.])
corners = np.stack([xs, ys]).T
hull = ConvexHull(corners)
print(hull.volume)

plt.plot(corners[hull.vertices,0], corners[hull.vertices,1], 'r--', lw=2)
plt.plot(corners[hull.vertices[0],0], corners[hull.vertices[0],1], 'ro')
plt.show()

In [4]:
EXP_PREFIX = "../../../data/local"
EXP_NAME = "ppo-point-embed/ppo_point_embed_2018_07_16_13_34_30_0056"
# EXP_NAME = "ppo-point-embed/ppo_point_embed_2018_07_16_13_34_30_0064"
# EXP_NAME = "ppo-point-embed/ppo_point_embed_2018_07_16_13_34_30_0155"
# EXP_NAME = "ppo-point-embed/ppo_point_embed_2018_07_16_13_34_30_0195"
# EXP_NAME = "ppo-point-embed/ppo_point_embed_2018_07_16_13_34_30_0243"
# EXP_NAME = "ppo-point-embed/ppo_point_embed_2018_07_16_13_34_30_0235"
# EXP_NAME = "ppo-point-embed/ppo_point_embed_2018_07_16_13_34_30_0163"

# EXP_NAME = "ppo-point-embed/ppo_point_embed_2018_07_16_13_34_30_0154"
# EXP_NAME = "ppo-point-embed/ppo_point_embed_2018_07_13_11_01_54_0019"
# EXP_NAME = "ppo-point-embed/ppo_point_embed_2018_07_13_11_01_54_0202"
# EXP_NAME = "ppo-point-embed/ppo_point_embed_2018_07_13_11_01_54_0283"
# EXP_NAME = "ppo-point-embed/ppo_point_embed_2018_07_13_11_01_54_0200"
# EXP_NAME = "ppo-point-embed/ppo_point_embed_2018_07_16_13_34_30_0194"
# EXP_NAME = "ppo-point-embed/ppo_point_embed_2018_07_16_13_34_30_0127"

# EXP_NAME = "ppo-point-embed-compose/ppo_point_embed_compose_2018_07_25_16_26_51_0001"

EXP_NAME = "sawyer-reach-embed/sawyer_reach_embed_2018_07_31_16_28_45_0001"

EXP_ITR = "70"
EXP = osp.join(EXP_PREFIX, EXP_NAME, "itr_{}.pkl".format(EXP_ITR))
try:
    sess.close()
except NameError:
    pass
sess = tf.InteractiveSession()

# Unpack the snapshot
snapshot = joblib.load(EXP)
env = snapshot["env"]
policy = snapshot["policy"]

# Tasks and goals
num_tasks = policy.task_space.flat_dim
task_envs = env._wrapped_env.env._task_envs
goals = np.array([te._goal for te in task_envs])
task_cmap = colormap_mpl(num_tasks)

# Embedding distributions
z_dists = [get_z_dist(t, policy) for t in range(num_tasks)]
z_means = np.array([d[0] for d in z_dists])
z_stds = np.array([d[1] for d in z_dists])

# Embedding efficiency
means_area = ConvexHull(z_means).volume
stds_area = np.sum(np.pi * z_stds.prod(axis=1))
print(stds_area/means_area)

FailedPreconditionError: Attempting to use uninitialized value vf/output_log_std/param
	 [[Node: _retval_vf/output_log_std/param_0_6 = _Retval[T=DT_FLOAT, index=6, _device="/job:localhost/replica:0/task:0/device:CPU:0"](vf/output_log_std/param)]]

In [ ]:
# Embedding visualization
plt.figure()
ax = plt.subplot(111, aspect='equal')
# centers and std circles
for t in range(num_tasks):
    mu = z_means[t, :]
    theta = z_stds[t, :]
    plt.scatter(x=mu[0], y=mu[1], color=task_cmap[t])
    e = Ellipse(tuple(mu), 3*theta[0], 3*theta[1], 0, color=task_cmap[t])
    e.set_alpha(0.1)
    ax.add_artist(e)
plt.grid(True)

In [ ]:
# Rollout visualization
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.figure(figsize=(8, 8))

MAX_PATH_LENGTH = 50
SAMPLING_POSITIONS = np.linspace(-5, 5, num=20)

for t in range(num_tasks):
    # Plot goal
    plt.scatter(
        [goals[t, 0]], [goals[t, 1]],
        #s=50,
        color=task_cmap[t],
        zorder=2,
        label="Task {}".format(t + 1))

    # Plot rollouts for linearly interpolated latents
    for i, x in enumerate(SAMPLING_POSITIONS):
        # systematic sampling of latent from embedding distribution
        z = z_means[t] + x * z_stds[t]

        # Run rollout
        obs = rollout(
            task_envs[t],
            policy,
            z,
            max_path_length=MAX_PATH_LENGTH,
            animated=False)

        # Plot rollout
        plt.plot(obs[:, 0], obs[:, 1], alpha=0.7, color=task_cmap[t])

plt.grid(True)
plt.xlim([-4, 4])
plt.ylim([-4, 4])
plt.axes().set_aspect('equal')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Rollout visualization
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.figure(figsize=(8, 8))

MAX_PATH_LENGTH = 50
SAMPLING_POSITIONS = np.linspace(0.0, 1.0, num=20)
# SAMPLING_POSITIONS = [0., 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# Plot goals
for t in range(num_tasks):
    # Plot goal
    plt.scatter(
        [goals[t, 0]], [goals[t, 1]],
        #s=50,
        color=task_cmap[t],
        zorder=2,
        label="Task {}".format(t + 1))

for t in [0, 2]:
    # Plot rollouts for linearly interpolated latents
    for i, x in enumerate(SAMPLING_POSITIONS):
        # systematic sampling of latent from embedding distribution
        z = (x * z_means[t] + (1.0 - x) * z_means[t+1])
        color = (x * task_cmap[t]) + ((1.0 - x) * task_cmap[t+1])

        # Run rollout
        obs = rollout(
            task_envs[t],
            policy,
            z,
            max_path_length=MAX_PATH_LENGTH,
            animated=False)

        # Plot rollout
        plt.plot(obs[:, 0], obs[:, 1], alpha=0.7, color=color)

plt.grid(True)
plt.xlim([-4, 4])
plt.ylim([-4, 4])
plt.axes().set_aspect('equal')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
EXP_PREFIX = "../../../data/local/ppo-point-embed"
EXP_ITR = "400"
EXP_NAMES = [
    'ppo_point_embed_2018_07_16_13_34_30_0064',
    'ppo_point_embed_2018_07_13_11_01_54_0123',
    'ppo_point_embed_2018_07_16_13_34_30_0099',
    'ppo_point_embed_2018_07_16_13_34_30_0243',
    'ppo_point_embed_2018_07_16_13_34_30_0207',
    'ppo_point_embed_2018_07_13_11_01_54_0203',
    'ppo_point_embed_2018_07_13_11_01_54_0121',
    'ppo_point_embed_2018_07_13_11_01_54_0204',
    'ppo_point_embed_2018_07_13_11_01_54_0119',
    'ppo_point_embed_2018_07_16_13_34_30_0063',
    'ppo_point_embed_2018_07_16_13_34_30_0062',
    'ppo_point_embed_2018_07_16_13_34_30_0001',
    'ppo_point_embed_2018_07_16_13_34_30_0170',
    'ppo_point_embed_2018_07_13_11_01_54_0362',
    'ppo_point_embed_2018_07_13_11_01_54_0285',
    'ppo_point_embed_2018_07_13_11_01_54_0282',
    'ppo_point_embed_2018_07_13_11_01_54_0280',
    'ppo_point_embed_2018_07_13_11_01_54_0199',
    'ppo_point_embed_2018_07_16_13_34_30_0026',
    'ppo_point_embed_2018_07_16_13_34_30_0134',
    'ppo_point_embed_2018_07_16_13_34_30_0242',
    'ppo_point_embed_2018_07_16_13_34_30_0016',
    'ppo_point_embed_2018_07_16_13_34_30_0006',
    'ppo_point_embed_2018_07_13_11_01_54_0284',
    'ppo_point_embed_2018_07_13_11_01_54_0038',
    'ppo_point_embed_2018_07_16_13_34_30_0098',
    'ppo_point_embed_2018_07_13_11_01_54_0200',
    'ppo_point_embed_2018_07_13_11_01_54_0042',
    'ppo_point_embed_2018_07_16_13_34_30_0239',
    'ppo_point_embed_2018_07_13_11_01_54_0283',
    'ppo_point_embed_2018_07_16_13_34_30_0167',
    'ppo_point_embed_2018_07_13_11_01_54_0192',
    'ppo_point_embed_2018_07_13_11_01_54_0113',
    'ppo_point_embed_2018_07_16_13_34_30_0130',
    'ppo_point_embed_2018_07_16_13_34_30_0095',
    'ppo_point_embed_2018_07_13_11_01_54_0031',
    'ppo_point_embed_2018_07_13_11_01_54_0122',
    'ppo_point_embed_2018_07_13_11_01_54_0194',
    'ppo_point_embed_2018_07_16_13_34_30_0058',
    'ppo_point_embed_2018_07_13_11_01_54_0202',
    'ppo_point_embed_2018_07_16_13_34_30_0131',
    'ppo_point_embed_2018_07_16_13_34_30_0059',
    'ppo_point_embed_2018_07_13_11_01_54_0040',
    'ppo_point_embed_2018_07_13_11_01_54_0032',
    'ppo_point_embed_2018_07_16_13_34_30_0202',
    'ppo_point_embed_2018_07_16_13_34_30_0203',
    'ppo_point_embed_2018_07_13_11_01_54_0028',
    'ppo_point_embed_2018_07_13_11_01_54_0033',
    'ppo_point_embed_2018_07_13_11_01_54_0271',
    'ppo_point_embed_2018_07_13_11_01_54_0029',
]

for n in EXP_NAMES:
    pkl = osp.join(EXP_PREFIX, n, "itr_{}.pkl".format(EXP_ITR))
    var = osp.join(EXP_PREFIX, n, "variant.json".format(EXP_ITR))
    try:
        sess.close()
    except NameError:
        pass
    sess = tf.InteractiveSession()
    
    # Unpack the variant
    v = json.load(open(var))
    pec = v["policy_ent_coeff"]
    eec = v["embedding_ent_coeff"]
    icc = v["inference_ce_coeff"]

    # Unpack the snapshot
    snapshot = joblib.load(pkl)
    env = snapshot["env"]
    policy = snapshot["policy"]

    # Tasks and goals
    num_tasks = policy.task_space.flat_dim
    task_envs = env._wrapped_env.env._task_envs
    goals = np.array([te._goal for te in task_envs])
    task_cmap = colormap_mpl(num_tasks)

    # Embedding distributions
    z_dists = [get_z_dist(t, policy) for t in range(num_tasks)]
    z_means = np.array([d[0] for d in z_dists])
    z_stds = np.array([d[1] for d in z_dists])
    
    # Rollout visualization
    matplotlib.rcParams['pdf.fonttype'] = 42
    matplotlib.rcParams['ps.fonttype'] = 42
    plt.figure(figsize=(4, 4))

    MAX_PATH_LENGTH = 50
    SAMPLING_POSITIONS = np.linspace(-5, 5, num=20)

    for t in range(num_tasks):
        # Plot goal
        plt.scatter(
            [goals[t, 0]], [goals[t, 1]],
            #s=50,
            color=task_cmap[t],
            zorder=2,
            label="Task {}".format(t + 1))

        # Plot rollouts for linearly interpolated latents
        for i, x in enumerate(SAMPLING_POSITIONS):
            # systematic sampling of latent from embedding distribution
            z = z_means[t] + x * z_stds[t]

            # Run rollout
            obs = rollout(
                task_envs[t],
                policy,
                z,
                max_path_length=MAX_PATH_LENGTH,
                animated=False)

            # Plot rollout
            plt.plot(obs[:, 0], obs[:, 1], alpha=0.7, color=task_cmap[t])

    plt.grid(True)
    plt.xlim([-4, 4])
    plt.ylim([-4, 4])
    plt.axes().set_aspect('equal')
    plt.title("pec: {}, eec: {}, icc: {}".format(pec, eec, icc))
    plt.legend()
    plt.tight_layout()
    plt.show()